### Scripts used to run different metagenome binning tools

In [ ]:
#!/bin/bash
#SBATCH -J "binning"
#SBATCH -p gpu
#SBATCH -A scc_users
#SBATCH -t 01-00:00:00
#SBATCH -N 1
#SBATCH -n 1
#SBATCH -c 24
#SBATCH -G 2
#SBATCH --mem=128G

source ~/miniforge3/bin/activate
"/usr/bin/time -v vamb --outdir vamb_results --fasta single_pooled_final.contigs.fa --bamfiles bamfiles/*_sorted.bam -m 1000 --minfasta 200000 --cuda -p 24"

conda activate metabat2_env
"/usr/bin/time -v metabat2 -i sorted_pooled_final.contigs.fa -a abundances_gf_sorted.tsv -o metabat2_results/metabat2_results -t 24 -m 1500"
conda deactivate

conda activate semibin_env
"/usr/bin/time -v SemiBin2 generate_sequence_features_single -i single_pooled_final.contigs.fa -b bamfiles/*_sorted.bam -o semibin2_results --min-len 1000 -p 24"

"/usr/bin/time -v SemiBin2 train_self --data semibin2_results/data.csv --data-split semibin2_results/data_split.csv -o semibin2_results -p 24"

"/usr/bin/time -v SemiBin2 bin_short -i single_pooled_final.contigs.fa --model semibin2_results/model.pt --data semibin2_results/data.csv -o semibin2_results/bins --minfasta-kbs 200 --min-len 1000 -p 24"
conda deactivate

conda activate genomeface_env
"/usr/bin/time -v genomeface -i sorted_pooled_final.contigs.fa -o genomeface_results -a abundances_gf_sorted.tsv -g marker_hits -m 1000"
conda deactivate

conda activate taxvamb_env
"/usr/bin/time -v vamb bin taxvamb --fasta sorted_pooled_final.contigs.fa --outdir taxvamb_results --bamfiles bamfiles/*_sorted.bam --taxonomy metabuli_results/taxonomy_result.tsv -m 1000 --minfasta 200000 --cuda -p 24"

"/usr/bin/time -v vamb bin taxvamb --fasta sorted_pooled_final.contigs.fa --outdir taxvamb_results_notaxometer --bamfiles bamfiles/*_sorted.bam --taxonomy metabuli_results/taxonomy_result.tsv -m 1000 --minfasta 200000 --cuda -p 24 --no_predictor"
conda deactivate

conda activate comebin_env
"/usr/bin/time -v run_comebin.sh -a single_pooled_final.contigs.fa -o comebin_results -p bamfiles -t 24"
conda deactivate




